In [ ]:

import pandas as pd
import numpy as np
import ast
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# STEAM GAMES

### Primero cargo el dataframe

In [ ]:
steam_games = pd.read_json('./datasets/steam_games.json.gz',compression='gzip',lines=True)

In [ ]:
steam_games.info()

In [ ]:
steam_games.sample(10)

### Elimino las filas completamente nulas

In [ ]:

steam_games.dropna(how='all',inplace=True)

### Busco id nulos

In [ ]:

steam_games[steam_games['id'].isnull()]

### Se encontraron dos filas con id nulo, una solo posee valores de url y precio, ademas el url es de la store de steam por lo tanto se eliminara.
### La otra fila con id nulo estaba replicada de otra fila, ya que tiene los mismos valores, por esto tambien se eliminara

In [ ]:
steam_games[steam_games['title'] == 'Batman: Arkham City - Game of the Year Edition'] 

In [ ]:
steam_games.dropna(subset=['id'],inplace=True) # Elimino las 2 filas con id nulo

### Busco los duplicados en id

In [ ]:
duplicados = steam_games['id'].value_counts() > 1
steam_games[steam_games['id'].isin(duplicados[duplicados].index)]

### Se encontraron dos filas completamente duplicadas se eliminara una

In [ ]:
steam_games.drop_duplicates(subset='id',inplace=True,keep='first')

### Convierto el id a tipo entero

In [ ]:
steam_games['id'] = steam_games['id'].astype('int') # Paso el id a entero

In [ ]:
steam_games.isnull().sum()

In [ ]:
steam_games[steam_games['app_name'].isnull()]

### El unico juego que posee nulo en app_name y title, no pose ni desarrolador ni publicador, por esto se eliminara.

In [ ]:
# El unico juego que no tiene ni titulo ni app_name tampoco tiene desarrollador o publicador, solo pose genero tag y un url que lleva a ningun juego, por esto se eliminara esta fila
steam_games.dropna(subset='app_name',inplace=True)

### Se remplazara los nulos de developer por los de publisher

In [ ]:
steam_games['developer'].fillna(steam_games['publisher'],inplace=True)

### Se eliminaran las columnas que no seran usadas por el estudio.
* publisher
* title
* url
* discount_price
* reviews_url
* early_access


In [ ]:
steam_proc = steam_games.drop(columns=['publisher','title','url','reviews_url','early_access'])

### Veo que tags y genres son columnas con datos similares, por esto buscare los nulos en genres y en tags

In [ ]:
steam_proc[steam_proc['genres'].isna()]

In [ ]:
steam_proc[steam_proc['tags'].isna()]['genres']

### Ahora voy a eliminar los valores que no esten en alguna fila  de genres de la columna tag para tener solo los generos
### Luego voy a agregar valores de tag que no esten en genres 
### Luego voy a remplazar los nulos de genres con los valores de tags
### Por ultimo voy a eliminar la columna tags que ya no aporta informacion

In [ ]:
genres = set(item for val in steam_proc['genres'].dropna() for item in val)# Obtengo los valores unicos de genres
 
steam_proc['tags'] = steam_proc['tags'].apply(lambda x: [item for item in x if item in genres] if isinstance(x, list) else x) # Los elimino de tags

steam_proc['genres'].fillna(steam_proc['tags'],inplace=True) # Remplazo los nulos de genres con los valores de tags


def agregar_genres_tags(fila): # Agrego valores de tangs en genres cuando estos no esten en genres.
    genres = fila['genres']
    tags = fila['tags']
    if isinstance(tags,list) and isinstance(genres,list):
        for tag in tags:
            if tag not in genres:
                genres.append(tag)
    return genres
steam_proc['genres'] = steam_proc.apply(lambda fila:agregar_genres_tags(fila),axis=1)


steam_proc.drop(columns=['tags'],inplace=True)




### Genero dummies para la columna genres,  y luego eliminare esta ultima

In [ ]:

# Ahora voy a generar dummies para genres 
steam_proc['genres'] =  steam_proc['genres'].apply(lambda x:".".join(x) if isinstance(x, list) else x)
dummies = steam_proc['genres'].str.get_dummies(sep='.')
#dummies = dummies.groupby(dummies.columns, axis=1).sum()
steam_proc = pd.concat([steam_proc,dummies],axis=1)

# Elimino la columna genres
steam_proc.drop(columns='genres',inplace=True)


### Columna Price:
* Primero voy a poner en valor 0 los que tengan un str con Free

In [ ]:

steam_proc.loc[steam_proc['price'].str.contains("Free", na=False), 'price'] = 0.0

* Reviso los valores de price y Free to Play


In [ ]:
steam_proc.loc[steam_proc['price'].apply(lambda x: isinstance(x, str)), ['price','Free to Play']]

* Remplazo los  valores que tenian un string y un precio por el precio asociado

In [ ]:
# A los dos valores que incluye el precio como parte de un string lo remplazo con su valor de precio
steam_proc['price'].replace('Starting at $499.00',499.0,inplace=True)
steam_proc['price'].replace('Starting at $449.00',499.0,inplace=True)

* Cuando precio posee un string y Free to Play vale 1 remplazo por 0

In [ ]:

steam_proc.loc[(steam_proc['Free to Play'] == 1) & steam_proc['price'].apply(lambda x: isinstance(x, str)), ['price']] = 0.0


* Veo los valores nulos de price donde Free to Play vale 1

In [ ]:
steam_proc[pd.isna(steam_proc['price'])][['Free to Play']].value_counts()

* A esos valores los remplazo por 1

In [ ]:
steam_proc.loc[(steam_proc['Free to Play'] == 1) & (pd.isna(steam_proc['price'])), 'price'] = 0.0

* Veo la cantidad de nulos que quedaron en price respecto al total,y veo que no llega al 4% de nulos

In [ ]:
steam_proc[steam_proc['price'].isnull()].shape[0]/steam_proc.shape[0]

* Elimino el resto de nulos en price, al no poder realizarse mas un analisis , ya que no considero correcto remlplazar un precio por un premedio u otro valor

In [ ]:
# Elimino los valores de precio donde sean nulos o donde o donde no sean de tipo string
steam_proc = steam_proc[steam_proc['price'].apply(lambda x: isinstance(x, float))]
steam_proc.dropna(subset='price',inplace=True)


* Tenindo normalizada esta columna la convierto a flaot

In [ ]:
# Teniendo precio con formato numerico convierto la columna a float
steam_proc['price'] = steam_proc['price'].astype('float')

### Veo los nulos que aun quedaron

In [ ]:
steam_proc.isnull().sum()

In [ ]:
print(f'Porcentaje de nulos : {steam_proc.dropna(subset=["developer","release_date","specs"]).shape[0] / steam_proc.shape[0]}')

### Quedaron valores nulos en release_date, developer y specs que equivalen aproximadamente al 11% del dataframe. 
### Y al no poder hacer otro tratado se eliminaran

In [ ]:
# Voy a liminar el resto de valores nulos
steam_proc.dropna(subset=['developer','release_date','specs'],inplace=True)
steam_proc.isnull().sum()

### Convierto la columna release_date en datetime y genero la columna año

In [ ]:
# Conversion de tipos
steam_proc['release_date'] = pd.to_datetime(steam_proc['release_date'], errors='coerce')
# Genero una columna para el año

steam_proc['Year'] = steam_proc['release_date'].dt.year

### Por ultimo exporto como csv el proceso de ETL

In [ ]:
steam_proc.to_csv('./datasets/steam_games.csv',index=False)

# USERS REVIEWS

### Ahora voy a trabajar con el archivo user_reviews.json.gz, el cual guarda registros de los comentarios de los usuarios sobre distintos items


###  Primero realizo la carga

In [ ]:
user_reviews_gz = "./datasets/user_reviews.json.gz"
filas=[]
with gzip.open(user_reviews_gz, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

user_review = pd.DataFrame(filas)

In [ ]:
user_review

### Ahora veo que la columna reviews es una lista de diccionarios, donde cada item de la lista equivale a un producto distinto que ese usuario haya comendado, y cada cada diccionario guarda variables referentes al item y su posteo.
### Por esto primero voy a usar explode para duplicar las filas por cada item de la lista, y luego voy a generar un dataframe con el diccionario nuevo y lo concatenare a traves de las columnas con el el dataframe con filas duplicadas

In [ ]:
user_review_explode = user_review.explode('reviews') # Duplico las filas generando un diccionario por cada dicc#ionario en la lista
# Ahora concateno el dataframe original, con el dataframe generado a partir de transformar los diccionarios a pandas
user_review_explode = pd.concat([user_review_explode.drop(['reviews'],axis=1),user_review_explode['reviews'].apply(pd.Series)],axis=1)  

### Elimino columnas que no seran usadas para el estudio
* user_url
* funny
* last_edited
* helpful
* 0

In [ ]:
user_review_explode.drop(columns=['user_url','funny','helpful','last_edited',0],inplace=True)

In [ ]:
user_review_explode

### Ahora a partir de la columna reveiw se hara un analisis de sentimiento, generando una nueva columna.
### Este se llamara "sentiment_analysis"  y tendra 2 para sentimiento positivo 0 para negativo y 1 para neutral
### Habiendo hecho esto elimino la columna review

In [ ]:
#Hago el analisis de sentimiento en la columna review
nltk.download('vader_lexicon')
model_sentimiento = SentimentIntensityAnalyzer()


def analizador(review):
    # Obtener el puntaje de sentimiento usando SentimentIntensityAnalyzer
    sentimiento_score = model_sentimiento.polarity_scores(review)
    
    # Clasifico el sentimiento
    
    if review and not pd.isnull(review):
        if sentimiento_score['compound'] >= 0.05:
            return 2  # Sentimiento positivo
        elif sentimiento_score['compound'] <= -0.05:
            return 0  # Sentimiento negativo
        else:
            return 1  # Sentimiento neutral
    else:
        return 1

* Para aplicar la funcion los nulos tienen que ser vacio

In [ ]:
user_review_explode['review'].fillna('',inplace=True)

In [ ]:
user_review_explode['sentiment_analysis']  = user_review_explode['review'].apply(analizador)
user_review_explode.drop(columns='review',inplace=True)

* Grafico las el analisis por sentimiento de las reviews

In [ ]:
sns.histplot(data=user_review_explode,x='sentiment_analysis')

### Se buscan los nulos que quedaron

In [ ]:
user_review_explode.isnull().sum()

In [ ]:
user_review_explode[user_review_explode.isnull().any(axis=1)]

### Hay nulos pero coiciden con todas las columnas de reviews por esto se eliminaran

In [ ]:

user_review_explode.dropna(inplace=True)


In [ ]:
user_review_explode

### Por ultimo se exporta como csv

In [ ]:
user_review_explode.to_csv('./datasets/user_reviews.csv',index=False)

# Users Item

In [ ]:
def cargar_df(ruta, variable_anidada):
    '''Función que recibe una ruta de acceso a un archivo json anidado y carga la información en un
    DataFrame de Pandas'''
    fila = []
    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
      for line in archivo.readlines():
          fila.append(ast.literal_eval(line))

    df = pd.DataFrame(fila)                                                 
    df = df.explode(variable_anidada).reset_index()                         
    df = df.drop(columns="index")                                           
    df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)   
    df = df.drop(columns=variable_anidada)                                  

    return df

In [ ]:
user_items = cargar_df("./datasets/users_items.json.gz",'items')

In [ ]:
user_items.describe()

In [ ]:
#sns.boxplot(data=user_items,y='playtime_forever')
sns.boxplot(data=user_items,y='items_count')
plt.ylabel(0,2000)

In [ ]:
user_items.isnull().sum()

In [ ]:
user_items[user_items['item_id'].isna()]

# Todos los nulos hacen referencia a las mismas filas por esto seran eliminados

In [ ]:
user_items.dropna(inplace=True)

In [ ]:
user_items.to_csv('./datasets/users_item_proc.csv',index=False)

In [ ]:
with gzip.open('user_items_proc.csv.gz', 'wb') as f:
    user_items.to_csv(f, index=False, encoding='utf-8')

In [20]:
fila = []
with gzip.open('./datasets/users_items.json.gz', 'rt', encoding='MacRoman') as archivo:
      for line in archivo.readlines():
          fila.append(ast.literal_eval(line))

In [21]:
user_items_explode = pd.DataFrame(fila).explode('items')

In [24]:
with gzip.open('user_items.csv.gz', 'wb') as f:
    user_items_explode.to_csv(f, index=False, encoding='utf-8')